In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn



In [2]:
df = pd.read_csv('LoanPred-Dataset.csv', skiprows = 1) #skiprows skips the header

In [ ]:
df.head()

In [ ]:
df.drop(columns = ['Unnamed: 0', 'ID'], axis = 1, inplace= True )

In [ ]:
df['PAY_0'].value_counts()
# df['PAY_0'].unique()
# by running this,we have 14737 values that has '0' which isnt specified in the question

In [ ]:
df['PAY_2'].value_counts()

In [ ]:
# Check unique values for all the columns
{column: len(df[column].unique()) for column in df.columns}

In [ ]:
df.isnull().sum()

## Exploratory Data Analysis

#### Categorical Dataset

In [ ]:
df.head(10)

In [ ]:
sns.countplot(df['SEX'])
# more female took loan than male

In [ ]:
sns.countplot(df['EDUCATION'])
# more university grauduate took loan

In [ ]:
sns.countplot(df['MARRIAGE'])
# single people took more loan

In [ ]:
sns.countplot(df['default payment next month'])
# few people default payment

## Correlation Matrix

In [ ]:
# Checking the correlation of the data

corr = df.corr()
plt.figure(figsize=(15,10))
sns.heatmap(corr, annot = True, cmap= 'BuPu')

## Creation of New Dataset

In [3]:
# FIll the null values with zero
for column in ['BILL_AMT1','BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']:
    df[column]= df[column].fillna(0)
# df.isnull().sum()

In [4]:
# Created a column to cover the bill amount columns
df['total_Bill_Amt']= (df['BILL_AMT1'] + df['BILL_AMT2'] + df['BILL_AMT3'] + 
                        df['BILL_AMT4'] + df['BILL_AMT5'] + df['BILL_AMT6'])

# df.head(2)

In [7]:
# choosing selected variable (selVar)
selVar = ['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4',
              'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4',  'PAY_AMT5', 'PAY_AMT6', 
                        'LIMIT_BAL', 'SEX', 'EDUCATION', 'AGE', 'MARRIAGE']


### Training and Modelling

Therefore, we need to keep a slice of data from the entire model selection and training process, and save it for the final evaluation. This slice of data is the “final exam” to our model and the exam questions must not be seen by the model before. Precisely, this is the workflow of how the data is being used:

1. training dataset is used to train a few candidate models
2. validation dataset is used to evaluate the candidate models
3. one of the candidates is chosen
4. the chosen model is trained with a new training dataset
5. the trained model is evaluated with the test dataset

In [8]:
X = df[selVar]
y =df['default payment next month']
X.columns

Index(['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1',
       'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
       'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'LIMIT_BAL', 'SEX', 'EDUCATION', 'AGE', 'MARRIAGE'],
      dtype='object')

In [9]:
# Standardaize X with standard scaler
from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler() # instatiate the scaler
# X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns) # all X variables will have a mean of 0 and a unique variance
# print (X)

# OR

scaler = StandardScaler()
scaler.fit(X)
Standardize = scaler.transform(X) # this standardize the data and make all values fall between 0 and 1

In [10]:
# split data into train and test, for both features and target

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify = y)
print (X_train.shape, y_train.shape, X_test.shape,  y_test.shape ) # so to confirm the sharing formular

(24000, 23) (24000,) (6000, 23) (6000,)


In [11]:
# Further split the training data into train and the validation
 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=100) # 0.25 x 0.8 = 0.2
print (X_train.shape, y_train.shape, X_val.shape,  y_val.shape ) # so to confirm the sharing formular

(20400, 23) (20400,) (3600, 23) (3600,)


In [12]:
# Define and train the model

from sklearn.linear_model import LogisticRegression
loan_model = LogisticRegression()
loan_model.fit(X_train, y_train)



C:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [13]:
# Evaluate or Validated your prediction  
val_prediction = loan_model.predict(X_val)
print('validation prediction: ', val_prediction) # print out top few validated predictions
print('initial default nextmonth: ', y)   # print the top few actual prices from validation data and check it against validated prediction



validation prediction:  [0 0 0 ... 0 0 0]
initial default nextmonth:  0        1
1        1
2        0
3        0
4        0
        ..
29995    0
29996    0
29997    1
29998    1
29999    1
Name: default payment next month, Length: 30000, dtype: int64


In [14]:
# Compute the Mean Absolute Error of the model, if Satisfied, then proceed to testing the model
from sklearn.metrics import mean_absolute_error
val_mae = mean_absolute_error(y_val, val_prediction)
print('mean absolute error: ', val_mae)

mean absolute error:  0.215


In [15]:
# Once we are satisfied with the mean absolute error of the model, 
# we proceed to train the Logistic Regression model with the full training data  (X,y)
# to improve the accuracy of the model
# And test with our reserved testing datasets

loan_model_FT = LogisticRegression()
loan_model_FT.fit(X, y)


C:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [16]:
# Retraining the model and predicting wiht full data X and y 

test_prediction = loan_model_FT.predict(X_test) # instatiate the model
print ('Final Prediction: ', test_prediction)


Final Prediction:  [0 0 0 ... 0 0 0]


In [17]:
# Checking the mean absolute error of the model

test_mae = mean_absolute_error(y_test, test_prediction)
print('Mean Absolute Error of the final prediction: ', test_mae)

Mean Absolute Error of the final prediction:  0.22116666666666668


### Making a Predictive System

In [18]:
# We take a data from the original raw dataset, run it on the model to test if its output correspond with the Y
input = (120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000) # Note that we didnt include Y column.capitalize

# We need to change the input to numpy array
input_numpy = np.asarray(input)

# The model was trained with 30,000 dataset, so its expects 30,000 dataset to run
# We then need to reshape so that a single dataset can run
input_reshape =  input_numpy.reshape(1, -1)

# We cannot use raw data to test a mpdel, we need to standardize it. Hence we Standardize the input
final_input = scaler.transform(input_reshape)
print (final_input)

# Now predict what the label Y will be 
prediction = loan_model_FT.predict(final_input)
print('Test prediction:', prediction)

if (prediction [0] == 0):
    print ('The person is not diabetic')
else:
    print(' The person is diabetic')

[[ 1.06782215e+05  1.78234817e+00  1.80992130e+00  1.89943574e+00
   2.31794297e+01 -6.16451690e-01 -6.95614667e-01 -6.90983429e-01
  -6.77928676e-01 -6.72497269e-01 -6.63025635e-01 -6.07688790e-01
  -2.37793846e-01 -1.40585726e-01 -1.10962591e-01 -8.75198352e-02
  -1.00692662e-01 -2.93382058e-01 -1.28316068e+00  2.04120489e+03
   1.26294072e+03 -3.84969199e+00  3.82873148e+03]]
Test prediction: [1]
 The person is diabetic


C:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
